In [1]:
# 구글드라이브에 업로드한 데이터를 Colab에서 사용하기 위해서 Mount를 수행합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 수집한 데이터들중 원하는 가져오기 위해 키워드를 입력합니다.
query_keyword = input("분석을 위한 키워드를 입력하세요 : ")

분석을 위한 키워드를 입력하세요 : 코로나


In [3]:
tokenizer = input("사용을 원하는 형태소 분석기를 입력하세요.\n(e.g. kkma, komoran, hannanum, okt, mecab) : ")

사용을 원하는 형태소 분석기를 입력하세요.
(e.g. kkma, komoran, hannanum, okt, mecab) : komoran


In [4]:
import pandas as pd

# Mount한 뒤 구글드라이브에서 각자 저장한 경로를 찾아서, 입력한 키워드를 가지는 데이터를 불러옵니다.
base_path = "/content/drive/MyDrive/Colab Notebooks/dataset/"
# csv 저장할 때 사용된 index는 필요 없기 때문에 "Unnamed: 0" column을 제거합니다.
data = pd.read_csv(base_path + f"kci_paper_{query_keyword}.csv").drop(columns=["Unnamed: 0"])
data

,title,author,date,organization,abstract
0,코로나(COVID-19) 사태의 장기화로 인한 스트레스 수준은 모든 소비자가 동일할...,"이희정/Lee Hee-Jung1, 이한녕/Lee Han-Yeong",2020.0,1차의과학대학교2차의과대학교 학생만족센터,최근 코로나 19사태의 장기화에 따라 기존의 감염병 유행과는 비교도 안 될 만큼 개...
1,코로나 혹은 포스트 코로나 시대의 청소년 영화교육,"오세섭/Oh, Se Seop",2020.0,1중앙대학교 영화학 박사,코로나19의 대유행으로 기존의 교육체계가 흔들리고 있다. 영화교육도 마찬가지다. 방...
2,코로나 19의 생명정치: 안전장치와 규율의 작동으로 바라본 한국의 코로나 19 ‘정...,박위준/Weejun Park,2020.0,1독립연구자,본 논문은 코로나 19를 관리하고 있는 한국의 방역 모델을 푸코의 생명정치 개념으로...
3,코로나 19로 인한 사회복지사의 일상생활 기능방해 정도가 우울에 미치는 영향: 코로...,"윤명숙/Yoon Myeong Sook1, 김요섭/Kim Yoseb",2020.0,1전북대학교2전북대학교,본 연구는 코로나 19 재난상황에서 증가한 사회복지서비스를 공공·민간 영역에서 전달...
4,코로나위기와 헌법국가 -독일에서의 코로나위기 대응에 대한 헌법적 논의를 중심으로-,"정문식/Jeong, Mun Sik1, 정호경/JUNG， HO KYOUNG",2020.0,1한양대학교2한양대학교,2020년 3월부터 본격적으로 독일에서 시작된 코로나19 감염의 확산으로 독일연방의...
...,...,...,...,...,...
3001,구전신화 속 세계창조 서사의 시원적 생태론과 그 의미- 함경도 망묵굿 <산천굿>을 ...,신동흔/Donghun Shin,2020.0,1건국대학교,이 논문에서는 최근 새롭게 보고된 최복녀본과 이찬엽 <산천굿> 자료를 바탕으로 한국...
3002,성찰 태도 촉진을 위한 플립러닝 기반의 비대면 수업 설계 및 효과 검증 － 표현적 ...,"장성민/Chang, Sung-min",2020.0,1인하대학교,이 연구는 표현적 쓰기 과제를 활용한 문제중심학습 상황에서 학습자의 성찰 태도를 촉...
3003,Sino-American Conflict during the Trump Admini...,"안인해/Ahn, Yinhay",2020.0,1고려대학교,The classic conflict between a rising power an...
3004,Simultaneous detection of major pathogens caus...,"김원일/Kim Won-Il1, 조용일2, 강석진3, 허태영4, 정영훈5, 김남수/N...",2012.0,1동물질병진단센터2농촌진흥청 국립축산과학원3농촌진흥청 국립축산과학원4농촌진흥청 국립...,Bovine diarrhea is a major economical burden t...


In [ ]:
# 형태소 분석을 위한 Konlpy를 다운로드 받습니다.
!pip install konlpy

In [ ]:
# konlpy에서 지원하는 형태소 분석기인 Mecab은 추가 설치가 필요합니다.
!curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
# 형태소 분석에 필요한 라이브러리들을 불러옵니다.
# konlpy에서 사용할 수 있는 형태소 분석기는 다음 5가지입니다.
from konlpy.tag import Kkma # 꼬꼬마
from konlpy.tag import Komoran # 코모란
from konlpy.tag import Hannanum # 한나눔
from konlpy.tag import Okt # Okt(Open Korean Text)
from konlpy.tag import Mecab # Mecab(미캅)

# 텍스트 전처리를 위한 정규표현식을 사용합니다.
import re

# 형태소 분석을 위한 객체 생성.
kkma = Kkma() # 꼬꼬마
komoran = Komoran() # 코모란
hannanum = Hannanum() # 한나눔
okt = Okt() # Okt
mecab = Mecab() # Mecab

In [ ]:
# 데이터의 초록은 "abstract" column에 저장되어 있어, 초록을 가져옵니다. 단, 초록이 빠져있는 row는 제외합니다.
text = data["abstract"].dropna()
print(text.iloc[0])
print()
print(text.iloc[1])
print()
print(len(text))

In [ ]:
def text_cleaning(doc):
    """
        텍스트 전처리 함수를 정의합니다.
    """
    # 이번 코드에서는 전처리로 한글을 제외한 모든 글자를 제거합니다. 전처리에는 정규표현식(Regular Expression)을 사용합니다.
    cleaned_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    return cleaned_doc

In [ ]:
def define_stopwords(path):
    """
    불용어를 처리하기 위해, 불용어 목록 데이터를 불러옵니다. 불러온 불용어들을 집합 자료구조에 저장합니다.
    불용어 목록 Source : https://www.ranks.nl/stopwords/korean
    
    """
    with open(base_path + "stopwords-ko.txt") as f:
        SW = set([word.strip() for word in f])
    
    return SW

In [ ]:
def text_tokenizing(doc, tokenizer):
    """
    주어진 텍스트를 tokenizing할 형태소 분석기를 선택합니다.
    """
    #tokenizer = tokenizer.eval
    #tokenized_doc = exec("%s.morphs(doc)" % tokenizer)
    if tokenizer == "kkma":
        tokenized_doc = [token for token in kkma.morphs(doc) if token not in SW and len(token) > 1]

    elif tokenizer == "komoran":
        tokenized_doc = [token for token in komoran.morphs(doc) if token not in SW and len(token) > 1]

    elif tokenizer == "hannanum":
        tokenized_doc = [token for token in hannanum.morphs(doc) if token not in SW and len(token) > 1]

    elif tokenizer == "okt":
        tokenized_doc = [token for token in okt.morphs(doc) if token not in SW and len(token) > 1]

    elif tokenizer == "mecab":
        tokenized_doc = [token for token in mecab.morphs(doc) if token not in SW and len(token) > 1]

    return tokenized_doc

In [ ]:
SW = define_stopwords(base_path + "stopwords-ko.txt")

In [ ]:
tokenized_docs = []
for idx, doc in enumerate(text):
    temp = text_cleaning(doc).strip()
    try:
        tokenized_docs.append(text_tokenizing(temp, tokenizer))
    except:
        print(f"{idx}th doc : ", temp)

print(tokenized_docs[0])
print("Total %d docs." % len(tokenized_docs))

In [ ]:
import pickle

with open(base_path + f"tokenized_docs({query_keyword}).pk", "wb") as f:
    pickle.dump(tokenized_docs, f)